In [1]:
import requests
from bs4 import BeautifulSoup
import re
from playwright.async_api import async_playwright
import pandas as pd
import time

In [2]:
df = pd.read_csv("../input/bpd.csv")
df.head()

,ia_number,incident_type,received_date,year,title_rank,first_name,last_name,keyname,race,gender,name_known,badge_number,allegation,finding,outcome
0,I2010-001,II,1/5/10,2010.0,Police Officer,Carlos A,Martinez,MartinezCarlosA,H,M,y,2244,Reporting for Duty,Sustained,Oral Reprimand
1,I2010-003,II,1/5/10,2010.0,Detective,Molwyn A,Shaw,ShawMolwynA,B,M,y,983,Reporting for Duty,Sustained,Oral Reprimand
2,E2010-002,CC,1/6/10,2010.0,Sergeant Detective,John J,Fitzgerald,FitzgeraldJohnJ,W,M,y,115,Neg.Duty/Unreasonable Judge,Exonerated,NaN
3,E2010-002,CC,1/6/10,2010.0,Sergeant,Edwin,Guzman,GuzmanEdwin,H,M,y,6595,Neg.Duty/Unreasonable Judge,Exonerated,NaN
4,E2010-002,CC,1/6/10,2010.0,Police Officer,Lynwood,Jenkins,JenkinsLynwood,B,M,y,1218,Neg.Duty/Unreasonable Judge,Exonerated,NaN


In [3]:
df1 = pd.read_csv("../input/2018.csv", encoding="latin1")
df1["year"] = 2018
df1.head()

,NAME,DEPARTMENT_NAME,TITLE,REGULAR,RETRO,OTHER,OVERTIME,INJURED,DETAIL,QUINN/EDUCATION INCENTIVE,TOTAL EARNINGS,POSTAL,year
0,"Williams,David C.",Boston Police Department,Police Officer,"95,184.68",NaN,"503,025.78","45,176.88","5,307.62","24,610.00","4,375.69","677,680.65",02356,2018
1,"Earley,John W",Boston Police Department,Police Lieutenant,"26,130.54","13,001.47","204,120.61",668.36,NaN,440.00,"6,532.63","250,893.61",02131,2018
2,"Bucelewicz,Helen",Boston Police Department,Police Officer,"8,676.89",NaN,"200,000.00",NaN,NaN,NaN,NaN,"208,676.89",02135,2018
3,"Evans,William B",Boston Police Department,Commissioner (Bpd),"149,999.93","3,461.56","165,858.44",NaN,NaN,NaN,NaN,"319,319.93",02127,2018
4,"Jones,Ronnie C",Boston Police Department,Police Officer,"462,149.98",NaN,"114,973.46",NaN,NaN,NaN,NaN,"577,123.44",02126,2018


In [4]:
df2 = pd.read_csv("../input/2019.csv", encoding="latin1")
df2["year"] = 2019
df2.head()

,NAME,DEPARTMENT_NAME,TITLE,REGULAR,RETRO,OTHER,OVERTIME,INJURED,DETAIL,QUINN/EDUCATION INCENTIVE,TOTAL EARNINGS,POSTAL,year
0,"Bottomley,Torii A",BPS Business Service,BPS Worker's Comp Job Class,-,-,-,-,"772,034.24",-,-,"772,034.24",1938,2019
1,"Smith,Lincoln",Workers Compensation Service,Workers Comp Job Classificatn,-,-,-,-,"401,182.80",-,-,"401,182.80",2125,2019
2,"Kervin,Timothy M.",Boston Police Department,Police Lieutenant/Hdq Dispatch,"142,061.86",-,"21,262.85","115,361.12",-,"41,360.00","35,492.87","355,538.70",2135,2019
3,"Danilecki,John H",Boston Police Department,Police Captain,"161,608.85",-,"24,040.29","68,964.13",-,"53,040.00","40,402.20","348,055.47",2081,2019
4,"Maguire,Joseph M",Boston Police Department,Police Sergeant/Hdq Dispatcher,"128,912.77",-,"7,128.30","121,616.21",-,"55,544.00","31,310.86","344,512.14",2038,2019


In [5]:
df1.columns = df1.columns.str.lower()
df1.columns = df1.columns.str.replace(" ", "_")
df1.head()

,name,department_name,title,regular,retro,other,overtime,injured,detail,quinn/education_incentive,total_earnings,postal,year
0,"Williams,David C.",Boston Police Department,Police Officer,"95,184.68",NaN,"503,025.78","45,176.88","5,307.62","24,610.00","4,375.69","677,680.65",02356,2018
1,"Earley,John W",Boston Police Department,Police Lieutenant,"26,130.54","13,001.47","204,120.61",668.36,NaN,440.00,"6,532.63","250,893.61",02131,2018
2,"Bucelewicz,Helen",Boston Police Department,Police Officer,"8,676.89",NaN,"200,000.00",NaN,NaN,NaN,NaN,"208,676.89",02135,2018
3,"Evans,William B",Boston Police Department,Commissioner (Bpd),"149,999.93","3,461.56","165,858.44",NaN,NaN,NaN,NaN,"319,319.93",02127,2018
4,"Jones,Ronnie C",Boston Police Department,Police Officer,"462,149.98",NaN,"114,973.46",NaN,NaN,NaN,NaN,"577,123.44",02126,2018


In [6]:
df18_prepped = df1[df1.department_name == "Boston Police Department"]

In [7]:
df2.columns = df2.columns.str.lower()
df2.columns = df2.columns.str.strip()
df2.columns = df2.columns.str.replace(" ", "_")
df2.head()

,name,department_name,title,regular,retro,other,overtime,injured,detail,quinn/education_incentive,total_earnings,postal,year
0,"Bottomley,Torii A",BPS Business Service,BPS Worker's Comp Job Class,-,-,-,-,"772,034.24",-,-,"772,034.24",1938,2019
1,"Smith,Lincoln",Workers Compensation Service,Workers Comp Job Classificatn,-,-,-,-,"401,182.80",-,-,"401,182.80",2125,2019
2,"Kervin,Timothy M.",Boston Police Department,Police Lieutenant/Hdq Dispatch,"142,061.86",-,"21,262.85","115,361.12",-,"41,360.00","35,492.87","355,538.70",2135,2019
3,"Danilecki,John H",Boston Police Department,Police Captain,"161,608.85",-,"24,040.29","68,964.13",-,"53,040.00","40,402.20","348,055.47",2081,2019
4,"Maguire,Joseph M",Boston Police Department,Police Sergeant/Hdq Dispatcher,"128,912.77",-,"7,128.30","121,616.21",-,"55,544.00","31,310.86","344,512.14",2038,2019


In [8]:
df19_prepped = df2[df2.department_name == "Boston Police Department"]

In [9]:
df20 = pd.read_csv("../input/2020.csv", encoding="latin1")
df20.head()

,NAME,DEPARTMENT_NAME,TITLE,REGULAR,RETRO,OTHER,OVERTIME,INJURED,DETAIL,QUINN / EDUCATION INCENTIVE,TOTAL EARNINGS,POSTAL
0,"Smith,Sean P",Boston Police Department,Police Lieutenant,"$145,787.56",NaN,"$13,160.94","$124,395.57",NaN,"$45,210.00","$36,447.09","$365,001.16",02186
1,"Lee,Waiman",Boston Police Department,Police Detective,"$108,803.25",NaN,"$16,324.27","$96,126.59",NaN,"$111,688.00","$27,200.78","$360,142.89",02134
2,"Brown,John M",Boston Police Department,Police Sergeant (Det),"$132,699.44",NaN,"$19,672.83","$155,123.11",NaN,"$5,088.00","$33,175.06","$345,758.44",02136
3,"Danilecki,John H",Boston Police Department,Police Captain,"$162,770.15",NaN,"$24,504.13","$64,326.60","$3,548.13","$43,200.00","$41,579.47","$339,928.48",02081
4,"Barrett,Thomas E.",Boston Police Department,Police Sergeant (Det),"$132,699.44",NaN,"$16,723.95","$156,754.23",NaN,NaN,"$33,175.06","$339,352.68",02132


In [10]:
df20["year"] = 2020
df20.columns = df20.columns.str.lower()
df20.columns = df20.columns.str.strip()
df20.columns = df20.columns.str.replace(" ", "_")
df20.head()


,name,department_name,title,regular,retro,other,overtime,injured,detail,quinn_/_education_incentive,total_earnings,postal,year
0,"Smith,Sean P",Boston Police Department,Police Lieutenant,"$145,787.56",NaN,"$13,160.94","$124,395.57",NaN,"$45,210.00","$36,447.09","$365,001.16",02186,2020
1,"Lee,Waiman",Boston Police Department,Police Detective,"$108,803.25",NaN,"$16,324.27","$96,126.59",NaN,"$111,688.00","$27,200.78","$360,142.89",02134,2020
2,"Brown,John M",Boston Police Department,Police Sergeant (Det),"$132,699.44",NaN,"$19,672.83","$155,123.11",NaN,"$5,088.00","$33,175.06","$345,758.44",02136,2020
3,"Danilecki,John H",Boston Police Department,Police Captain,"$162,770.15",NaN,"$24,504.13","$64,326.60","$3,548.13","$43,200.00","$41,579.47","$339,928.48",02081,2020
4,"Barrett,Thomas E.",Boston Police Department,Police Sergeant (Det),"$132,699.44",NaN,"$16,723.95","$156,754.23",NaN,NaN,"$33,175.06","$339,352.68",02132,2020


In [11]:
df20.columns == df18_prepped.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True])

In [12]:
df20.rename(columns={"quinn_/_education_incentive": "quinn/education_incentive"}, inplace=True)

In [13]:
df20_prepped = df20[df20.department_name == "Boston Police Department"]

In [14]:
df21 = pd.read_csv("../input/2021.csv", encoding="latin1")
df21.head()

,NAME,DEPARTMENT_NAME,TITLE,REGULAR,RETRO,OTHER,OVERTIME,INJURED,DETAIL,QUINN_EDUCATION_INCENTIVE,TOTAL_GROSS,POSTAL
0,"Beckers,Richard",Boston Police Department,Police Officer,NaN,NaN,"1,264,843.63",NaN,NaN,NaN,NaN,"1,264,843.63",02119
1,"McGowan,Jacqueline M.",Boston Police Department,Police Officer,NaN,NaN,"1,252,990.81",NaN,NaN,NaN,NaN,"1,252,990.81",02129
2,"Harris,Shawn N",Boston Police Department,Police Offc Comm Serv Offc 3$8,"69,772.10",NaN,"212,739.48","82,300.87","30,939.24","12,144.00","25,178.06","433,073.75",02130
3,"Washington,Walter",Boston Police Department,Police Officer,"100,963.38",NaN,"211,900.28","67,849.66",NaN,"9,016.00","10,096.55","399,825.87",02368
4,"Mosley Jr.,Curtis",Boston Police Department,Police Offc Comm Serv Offc 3$8,"109,858.02",NaN,"192,097.54","75,938.65",NaN,"19,550.00",NaN,"397,444.21",02301


In [15]:
df21["year"] = 2021
df21.columns = df21.columns.str.lower()
df21.columns = df21.columns.str.strip()
df21.columns = df21.columns.str.replace(" ", "_")
df21.head()

,name,department_name,title,regular,retro,other,overtime,injured,detail,quinn_education_incentive,total_gross,postal,year
0,"Beckers,Richard",Boston Police Department,Police Officer,NaN,NaN,"1,264,843.63",NaN,NaN,NaN,NaN,"1,264,843.63",02119,2021
1,"McGowan,Jacqueline M.",Boston Police Department,Police Officer,NaN,NaN,"1,252,990.81",NaN,NaN,NaN,NaN,"1,252,990.81",02129,2021
2,"Harris,Shawn N",Boston Police Department,Police Offc Comm Serv Offc 3$8,"69,772.10",NaN,"212,739.48","82,300.87","30,939.24","12,144.00","25,178.06","433,073.75",02130,2021
3,"Washington,Walter",Boston Police Department,Police Officer,"100,963.38",NaN,"211,900.28","67,849.66",NaN,"9,016.00","10,096.55","399,825.87",02368,2021
4,"Mosley Jr.,Curtis",Boston Police Department,Police Offc Comm Serv Offc 3$8,"109,858.02",NaN,"192,097.54","75,938.65",NaN,"19,550.00",NaN,"397,444.21",02301,2021


In [16]:
df21.rename(columns={"quinn_education_incentive": "quinn/education_incentive"}, inplace=True)
df21.rename(columns={"total_gross": "total_earnings"}, inplace=True)

In [17]:
df21.columns == df18_prepped.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [18]:
df21_prepped = df21[df21.department_name == "Boston Police Department"]

In [19]:
df18_21 = pd.concat([df18_prepped, df19_prepped, df20_prepped, df21_prepped])
df18_21.head()

,name,department_name,title,regular,retro,other,overtime,injured,detail,quinn/education_incentive,total_earnings,postal,year
0,"Williams,David C.",Boston Police Department,Police Officer,"95,184.68",NaN,"503,025.78","45,176.88","5,307.62","24,610.00","4,375.69","677,680.65",02356,2018
1,"Earley,John W",Boston Police Department,Police Lieutenant,"26,130.54","13,001.47","204,120.61",668.36,NaN,440.00,"6,532.63","250,893.61",02131,2018
2,"Bucelewicz,Helen",Boston Police Department,Police Officer,"8,676.89",NaN,"200,000.00",NaN,NaN,NaN,NaN,"208,676.89",02135,2018
3,"Evans,William B",Boston Police Department,Commissioner (Bpd),"149,999.93","3,461.56","165,858.44",NaN,NaN,NaN,NaN,"319,319.93",02127,2018
4,"Jones,Ronnie C",Boston Police Department,Police Officer,"462,149.98",NaN,"114,973.46",NaN,NaN,NaN,NaN,"577,123.44",02126,2018


In [20]:
df18_21.fillna(0, inplace=True)
df18_21.head()

,name,department_name,title,regular,retro,other,overtime,injured,detail,quinn/education_incentive,total_earnings,postal,year
0,"Williams,David C.",Boston Police Department,Police Officer,"95,184.68",0,"503,025.78","45,176.88","5,307.62","24,610.00","4,375.69","677,680.65",02356,2018
1,"Earley,John W",Boston Police Department,Police Lieutenant,"26,130.54","13,001.47","204,120.61",668.36,0,440.00,"6,532.63","250,893.61",02131,2018
2,"Bucelewicz,Helen",Boston Police Department,Police Officer,"8,676.89",0,"200,000.00",0,0,0,0,"208,676.89",02135,2018
3,"Evans,William B",Boston Police Department,Commissioner (Bpd),"149,999.93","3,461.56","165,858.44",0,0,0,0,"319,319.93",02127,2018
4,"Jones,Ronnie C",Boston Police Department,Police Officer,"462,149.98",0,"114,973.46",0,0,0,0,"577,123.44",02126,2018


In [21]:
df18_21.replace("-", 0, inplace=True)
df18_21.head()

,name,department_name,title,regular,retro,other,overtime,injured,detail,quinn/education_incentive,total_earnings,postal,year
0,"Williams,David C.",Boston Police Department,Police Officer,"95,184.68",0,"503,025.78","45,176.88","5,307.62","24,610.00","4,375.69","677,680.65",02356,2018
1,"Earley,John W",Boston Police Department,Police Lieutenant,"26,130.54","13,001.47","204,120.61",668.36,0,440.00,"6,532.63","250,893.61",02131,2018
2,"Bucelewicz,Helen",Boston Police Department,Police Officer,"8,676.89",0,"200,000.00",0,0,0,0,"208,676.89",02135,2018
3,"Evans,William B",Boston Police Department,Commissioner (Bpd),"149,999.93","3,461.56","165,858.44",0,0,0,0,"319,319.93",02127,2018
4,"Jones,Ronnie C",Boston Police Department,Police Officer,"462,149.98",0,"114,973.46",0,0,0,0,"577,123.44",02126,2018


In [22]:
df18_21.to_csv("../data_prep/payroll18_21.csv", index=False)

In [23]:
df11 = pd.read_csv("../input/2011.csv", encoding="latin1")
df11.head()

,Name,Department Name,Title,Regular,Retro,Other,Overtime,Injured,Detail,Quinn,Total Earnings,Zip Code
0,"Abadi,Kidani A",Assessing Department,Property Officer (Asn),$33065.38,$0.00,$0.00,$379.49,$0.00,$0.00,$0.00,$33444.87,02118-3126
1,"Ablon,Jordan N",ASD Office Of Labor Relation,Asst Corp Counsel III,$76051.24,$0.00,$1321.03,$0.00,$0.00,$0.00,$0.00,$77372.27,02135-5943
2,"Accardi,Patricia",Transportation-Parking Clerk,Chief Claims Investigator,$56430.79,$0.00,$0.00,$418.58,$0.00,$0.00,$0.00,$56849.37,02081-3751
3,"Ackerly,Lyn E.",Boston Public Library,Spec Library Asst I,$35058.78,$0.00,$0.00,$439.19,$0.00,$0.00,$0.00,$35497.97,02118-0000
4,"Adams,Carey L.",Law Department,Prin Clerk,$41588.83,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$41588.83,02131-4834


In [24]:
df12 = pd.read_csv("../input/2012.csv", encoding="latin1")
df12.head()

,NAME,DEPARTMENT,TITLE,REGULAR,RETRO,OTHER,OVERTIME,INJURED,DETAIL,QUINN,TOTAL EARNINGS,ZIP
0,"Perch,Rhonda N.",Administration and Finance,Admin Secretary,$43646.22,$199.52,$1120.82,$0.00,$0.00,$0.00,$0.00,$44966.56,02124-3735
1,"Harris,Sonya L.",Administration and Finance,Data Proc Systems Analyst,$72115.30,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$72115.30,02124-3722
2,"Gallagher,Anthony J.",Administration and Finance,Exec Asst,$8828.56,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$8828.56,02113-1172
3,"Natoli,John F.",Administration and Finance,Exec Asst (Obpe),$100486.52,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$100486.52,02108-3407
4,"Quirk,Devin Lyons",Administration and Finance,Exec Asst (Obpe),$84853.21,$0.00,$1629.36,$0.00,$0.00,$0.00,$0.00,$86482.57,02129-3728


In [25]:
df11["year"] = 2011
df11.columns = df11.columns.str.lower()
df11.columns = df11.columns.str.strip()
df11.columns = df11.columns.str.replace(" ", "_")
df11.head()

,name,department_name,title,regular,retro,other,overtime,injured,detail,quinn,total_earnings,zip_code,year
0,"Abadi,Kidani A",Assessing Department,Property Officer (Asn),$33065.38,$0.00,$0.00,$379.49,$0.00,$0.00,$0.00,$33444.87,02118-3126,2011
1,"Ablon,Jordan N",ASD Office Of Labor Relation,Asst Corp Counsel III,$76051.24,$0.00,$1321.03,$0.00,$0.00,$0.00,$0.00,$77372.27,02135-5943,2011
2,"Accardi,Patricia",Transportation-Parking Clerk,Chief Claims Investigator,$56430.79,$0.00,$0.00,$418.58,$0.00,$0.00,$0.00,$56849.37,02081-3751,2011
3,"Ackerly,Lyn E.",Boston Public Library,Spec Library Asst I,$35058.78,$0.00,$0.00,$439.19,$0.00,$0.00,$0.00,$35497.97,02118-0000,2011
4,"Adams,Carey L.",Law Department,Prin Clerk,$41588.83,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$41588.83,02131-4834,2011


In [26]:
df12["year"] = 2012
df12.columns = df12.columns.str.lower()
df12.columns = df12.columns.str.strip()
df12.columns = df12.columns.str.replace(" ", "_")
df12.head()

,name,department,title,regular,retro,other,overtime,injured,detail,quinn,total_earnings,zip,year
0,"Perch,Rhonda N.",Administration and Finance,Admin Secretary,$43646.22,$199.52,$1120.82,$0.00,$0.00,$0.00,$0.00,$44966.56,02124-3735,2012
1,"Harris,Sonya L.",Administration and Finance,Data Proc Systems Analyst,$72115.30,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$72115.30,02124-3722,2012
2,"Gallagher,Anthony J.",Administration and Finance,Exec Asst,$8828.56,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$8828.56,02113-1172,2012
3,"Natoli,John F.",Administration and Finance,Exec Asst (Obpe),$100486.52,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$100486.52,02108-3407,2012
4,"Quirk,Devin Lyons",Administration and Finance,Exec Asst (Obpe),$84853.21,$0.00,$1629.36,$0.00,$0.00,$0.00,$0.00,$86482.57,02129-3728,2012


In [27]:
df11.columns == df12.columns

array([ True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True, False,  True])

In [28]:
df12.rename(columns={"department": "department_name"}, inplace=True)
df11.rename(columns={"zip_code": "zip"}, inplace=True)
df11.columns == df12.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [29]:
df11_prepped = df11[df11.department_name == "Boston Police Department"]
df12_prepped = df12[df12.department_name == "Boston Police Department"]

In [30]:
df12_prepped.shape

(3034, 13)

In [31]:
df12.department_name.unique()

array(['Administration and Finance', 'Arts & Cultural Development',
       'ASD Human Resources', 'ASD Intergvernmtl Relations',
       'ASD Office of Budget Mangmnt', 'ASD Office Of Labor Relation',
       'ASD Purchasing Division', 'Assessing Department',
       'Auditing Department', 'Boston City Council',
       'Boston Cntr - Youth & Families', 'Boston Public Library',
       'Cemetery Division', 'City Clerk',
       'Comm. for Persons with Disabil', 'Consumer Affairs & Licensing',
       'Dept of Voter Mobilization', 'Dpt of Innovation & Technology',
       'Elderly Commission', 'Emergency Management',
       'Environment Department', 'Finance Commission',
       'Inspectional Services Dept', 'Law Department', 'Licensing Board',
       "Mayor's Office", "Mayor's Office-Public Info",
       'Neighborhood Development', 'Neighborhood Services',
       'Ofc Boston Residents Job Pol', 'Ofc Chf Public Works Transport',
       'Office Of Civil Rights', 'Office of New Bostonians',
      

In [32]:
df13 = pd.read_csv("../input/2013.csv", encoding="latin1")
df13.head()

,NAME,TITLE,DEPARTMENT,REGULAR,RETRO,OTHER,OVERTIME,INJURED,DETAIL,QUINN,TOTAL EARNINGS,ZIP
0,"Ababio-Fernandez,Ruby A.",Principal Elementary,Boston Public Schools,$0.00,$0.00,$12122.88,$0.00,$0.00,$0.00,$0.00,$12122.88,11001-3647
1,"Abadi,Kidani A",Property Officer,Assessing Department,$37588.30,$0.00,$431.87,$0.00,$0.00,$0.00,$0.00,$38020.17,02118-3126
2,"Abasciano,Joseph",Police Officer,Boston Police Department,$66673.45,$0.00,$800.00,$20318.50,$3163.30,$890.00,$0.00,$91845.25,02132-3802
3,"Abban,Christopher John",Fire Fighter,Boston Fire Department,$87210.42,$0.00,$550.00,$12871.08,$0.00,$4798.00,$0.00,$105429.50,02132-3511
4,"Abbott,John R.",Teacher,Boston Public Schools,$88440.27,$836.63,$0.00,$0.00,$0.00,$0.00,$0.00,$89276.90,02445-2124


In [33]:
df13["year"] = 2013
df13.columns = df13.columns.str.lower()
df13.columns = df13.columns.str.strip()
df13.columns = df13.columns.str.replace(" ", "_")
df13.head()


,name,title,department,regular,retro,other,overtime,injured,detail,quinn,total_earnings,zip,year
0,"Ababio-Fernandez,Ruby A.",Principal Elementary,Boston Public Schools,$0.00,$0.00,$12122.88,$0.00,$0.00,$0.00,$0.00,$12122.88,11001-3647,2013
1,"Abadi,Kidani A",Property Officer,Assessing Department,$37588.30,$0.00,$431.87,$0.00,$0.00,$0.00,$0.00,$38020.17,02118-3126,2013
2,"Abasciano,Joseph",Police Officer,Boston Police Department,$66673.45,$0.00,$800.00,$20318.50,$3163.30,$890.00,$0.00,$91845.25,02132-3802,2013
3,"Abban,Christopher John",Fire Fighter,Boston Fire Department,$87210.42,$0.00,$550.00,$12871.08,$0.00,$4798.00,$0.00,$105429.50,02132-3511,2013
4,"Abbott,John R.",Teacher,Boston Public Schools,$88440.27,$836.63,$0.00,$0.00,$0.00,$0.00,$0.00,$89276.90,02445-2124,2013


In [34]:
df14 = pd.read_csv("../input/2014.csv", encoding="latin1")
df14.head()

,NAME,TITLE,DEPARTMENT NAME,REGULAR,RETRO,OTHER,OVERTIME,INJURED,DETAIL,QUINN,TOTAL EARNINGS,ZIP
0,"Adario,Anthony J",Supvising Claims Agent (Asd),ASD Human Resources,$98538.32,$0.00,$1842.87,$0.00,$0.00,$0.00,$0.00,$100381.19,02132
1,"Baker,Katherine E.",Nurse Case Manager,ASD Human Resources,$83942.21,$0.00,$225.00,$0.00,$0.00,$0.00,$0.00,$84167.21,02136
2,"Braga,Ann Hess",HumanResourcesComplianceOffcr,ASD Human Resources,$98538.34,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$98538.34,02126
3,"Calobrisi,Jane Ellen",Personnel Analyst,ASD Human Resources,$70073.81,$0.00,$225.00,$0.00,$0.00,$0.00,$0.00,$70298.81,02090
4,"Castillo-Cook,Wendolyn M.",Alcholism Coord I,ASD Human Resources,$79253.11,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$79253.11,02121


In [35]:
df14.columns = df14.columns.str.lower()
df14.columns = df14.columns.str.strip()
df14.columns = df14.columns.str.replace(" ", "_")
df14.head()

,name,title,department_name,regular,retro,other,overtime,injured,detail,quinn,total_earnings,zip
0,"Adario,Anthony J",Supvising Claims Agent (Asd),ASD Human Resources,$98538.32,$0.00,$1842.87,$0.00,$0.00,$0.00,$0.00,$100381.19,02132
1,"Baker,Katherine E.",Nurse Case Manager,ASD Human Resources,$83942.21,$0.00,$225.00,$0.00,$0.00,$0.00,$0.00,$84167.21,02136
2,"Braga,Ann Hess",HumanResourcesComplianceOffcr,ASD Human Resources,$98538.34,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$98538.34,02126
3,"Calobrisi,Jane Ellen",Personnel Analyst,ASD Human Resources,$70073.81,$0.00,$225.00,$0.00,$0.00,$0.00,$0.00,$70298.81,02090
4,"Castillo-Cook,Wendolyn M.",Alcholism Coord I,ASD Human Resources,$79253.11,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$79253.11,02121


In [36]:
df14["year"] = 2014
df14.columns == df13.columns

array([ True,  True, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [37]:
df13.rename(columns={"department": "department_name"}, inplace=True)
df13.columns == df14.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [38]:
df11.columns ==  df13.columns

array([ True, False, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [39]:
df11_12 = pd.concat([df11_prepped, df12_prepped])
df13_prepped = df13[df13.department_name == "Boston Police Department"]
df14_prepped = df14[df14.department_name == "Boston Police Department"]
df13_14 = pd.concat([df13_prepped, df14_prepped])


In [40]:
df11_12.columns == df13_14.columns

array([ True, False, False,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [41]:
df11_12.head()

,name,department_name,title,regular,retro,other,overtime,injured,detail,quinn,total_earnings,zip,year
3703,"Abasciano,Joseph",Boston Police Department,Police Officer,$68181.24,$0.00,$3763.50,$14177.65,$0.00,$4867.00,$0.00,$90989.39,02132-3802,2011
3704,"Abdul-Aziz,Ramadani",Boston Police Department,Police Officer,$73935.02,$0.00,$800.00,$5573.40,$0.00,$44865.00,$0.00,$125173.42,02119-1618,2011
3705,"Abichaker,Linda",Boston Police Department,Police Clerk And Typist,$34958.02,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$34958.02,02132-2737,2011
3706,"Abreu,Cesar",Boston Police Department,Police Officer,$69701.19,$0.00,$800.00,$37800.40,$0.00,$29826.00,$0.00,$138127.59,02136-1540,2011
3707,"Abron,Kim R.",Boston Police Department,School Traffic Supv,$10129.29,$0.00,$450.00,$0.00,$0.00,$0.00,$0.00,$10579.29,02124-4819,2011


In [42]:
df13_14.head()

,name,title,department_name,regular,retro,other,overtime,injured,detail,quinn,total_earnings,zip,year
2,"Abasciano,Joseph",Police Officer,Boston Police Department,$66673.45,$0.00,$800.00,$20318.50,$3163.30,$890.00,$0.00,$91845.25,02132-3802,2013
13,"Abdul-Aziz,Ramadani",Police Officer,Boston Police Department,$73105.70,$0.00,$800.00,$9666.15,$0.00,$38188.00,$0.00,$121759.85,02119-1618,2013
21,"Abichaker,Linda",Police Clerk And Typist,Boston Police Department,$39008.05,$0.00,$2500.00,$0.00,$0.00,$0.00,$0.00,$41508.05,02132-2737,2013
27,"Abrahamson,Patrick Olaf",Police Officer,Boston Police Department,$2065.38,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$2065.38,02135-2503,2013
33,"Abreu,Cesar",Police Officer,Boston Police Department,$69836.75,$0.00,$800.00,$64216.88,$0.00,$23668.00,$0.00,$158521.63,02136-1540,2013


In [43]:
temp_col = df13_14.title

In [44]:
df13_14.title = df13_14.department_name
df13_14.head()

,name,title,department_name,regular,retro,other,overtime,injured,detail,quinn,total_earnings,zip,year
2,"Abasciano,Joseph",Boston Police Department,Boston Police Department,$66673.45,$0.00,$800.00,$20318.50,$3163.30,$890.00,$0.00,$91845.25,02132-3802,2013
13,"Abdul-Aziz,Ramadani",Boston Police Department,Boston Police Department,$73105.70,$0.00,$800.00,$9666.15,$0.00,$38188.00,$0.00,$121759.85,02119-1618,2013
21,"Abichaker,Linda",Boston Police Department,Boston Police Department,$39008.05,$0.00,$2500.00,$0.00,$0.00,$0.00,$0.00,$41508.05,02132-2737,2013
27,"Abrahamson,Patrick Olaf",Boston Police Department,Boston Police Department,$2065.38,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$2065.38,02135-2503,2013
33,"Abreu,Cesar",Boston Police Department,Boston Police Department,$69836.75,$0.00,$800.00,$64216.88,$0.00,$23668.00,$0.00,$158521.63,02136-1540,2013


In [45]:
df13_14.rename(columns={"department_name": "title1"}, inplace=True)
df13_14.rename(columns={"title": "department_name"}, inplace=True)
df13_14.head()

,name,department_name,title1,regular,retro,other,overtime,injured,detail,quinn,total_earnings,zip,year
2,"Abasciano,Joseph",Boston Police Department,Boston Police Department,$66673.45,$0.00,$800.00,$20318.50,$3163.30,$890.00,$0.00,$91845.25,02132-3802,2013
13,"Abdul-Aziz,Ramadani",Boston Police Department,Boston Police Department,$73105.70,$0.00,$800.00,$9666.15,$0.00,$38188.00,$0.00,$121759.85,02119-1618,2013
21,"Abichaker,Linda",Boston Police Department,Boston Police Department,$39008.05,$0.00,$2500.00,$0.00,$0.00,$0.00,$0.00,$41508.05,02132-2737,2013
27,"Abrahamson,Patrick Olaf",Boston Police Department,Boston Police Department,$2065.38,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$2065.38,02135-2503,2013
33,"Abreu,Cesar",Boston Police Department,Boston Police Department,$69836.75,$0.00,$800.00,$64216.88,$0.00,$23668.00,$0.00,$158521.63,02136-1540,2013


In [46]:
df13_14.title1 = temp_col
df13_14.rename(columns={"title1": "title"}, inplace=True)
df13_14.head()

,name,department_name,title,regular,retro,other,overtime,injured,detail,quinn,total_earnings,zip,year
2,"Abasciano,Joseph",Boston Police Department,Police Officer,$66673.45,$0.00,$800.00,$20318.50,$3163.30,$890.00,$0.00,$91845.25,02132-3802,2013
13,"Abdul-Aziz,Ramadani",Boston Police Department,Police Officer,$73105.70,$0.00,$800.00,$9666.15,$0.00,$38188.00,$0.00,$121759.85,02119-1618,2013
21,"Abichaker,Linda",Boston Police Department,Police Clerk And Typist,$39008.05,$0.00,$2500.00,$0.00,$0.00,$0.00,$0.00,$41508.05,02132-2737,2013
27,"Abrahamson,Patrick Olaf",Boston Police Department,Police Officer,$2065.38,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$2065.38,02135-2503,2013
33,"Abreu,Cesar",Boston Police Department,Police Officer,$69836.75,$0.00,$800.00,$64216.88,$0.00,$23668.00,$0.00,$158521.63,02136-1540,2013


In [47]:
df13_14.columns == df11_12.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [48]:
df11_14 = pd.concat([df11_12, df13_14])
df11_14.to_csv("../data_prep/payroll11_14.csv", index=False)

In [49]:
df15 = pd.read_csv("../input/2015.csv", encoding="latin1")
df15.head()

,NAME,DEPARTMENT_NAME,TITLE,REGULAR,RETRO,OTHER,OVERTIME,INJURED,DETAILS,QUINN/EDUCATION INCENTIVE,TOTAL EARNINGS,POSTAL
0,"Abadi,Kidani A",Assessing Department,Property Officer,$44006.26,$0.00,$275.00,$324.12,$0.00,$0.00,$0.00,$44605.38,02118
1,"Abasciano,Joseph",Boston Police Department,Police Officer,$-886.20,$0.00,$8088.35,$544.83,$81996.78,$0.00,$20667.80,$110411.56,02132
2,"Abban,Christopher John",Boston Fire Department,Fire Fighter,$101468.29,$0.00,$550.00,$15146.51,$0.00,$5168.00,$0.00,$122332.80,02132
3,"Abbate-Vaughn,Jorgelina",BPS Ellis Elementary,Teacher,$42919.31,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$42919.31,02481
4,"Abberton,James P",Public Works Department,Maint Mech (Carpenter)##,$39768.17,$0.00,$126.00,$13974.97,$0.00,$0.00,$0.00,$53869.14,02127


In [50]:
df15.columns = df15.columns.str.lower()
df15.columns = df15.columns.str.strip()
df15.columns = df15.columns.str.replace(" ", "_")
df15.rename(columns={"details": "detail"}, inplace=True)
df15.head()

,name,department_name,title,regular,retro,other,overtime,injured,detail,quinn/education_incentive,total_earnings,postal
0,"Abadi,Kidani A",Assessing Department,Property Officer,$44006.26,$0.00,$275.00,$324.12,$0.00,$0.00,$0.00,$44605.38,02118
1,"Abasciano,Joseph",Boston Police Department,Police Officer,$-886.20,$0.00,$8088.35,$544.83,$81996.78,$0.00,$20667.80,$110411.56,02132
2,"Abban,Christopher John",Boston Fire Department,Fire Fighter,$101468.29,$0.00,$550.00,$15146.51,$0.00,$5168.00,$0.00,$122332.80,02132
3,"Abbate-Vaughn,Jorgelina",BPS Ellis Elementary,Teacher,$42919.31,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$42919.31,02481
4,"Abberton,James P",Public Works Department,Maint Mech (Carpenter)##,$39768.17,$0.00,$126.00,$13974.97,$0.00,$0.00,$0.00,$53869.14,02127


In [51]:
df16 = pd.read_csv("../input/2016.csv", encoding="latin1")
df16.head()

,NAME,DEPARTMENT_NAME,TITLE,REGULAR,RETRO,OTHER,OVERTIME,INJURED,DETAIL,QUINN/EDUCATION INCENTIVE,TOTAL EARNINGS,POSTAL
0,"Abadi,Kidani A",Assessing Department,Property Officer,"$46,291.98",NaN,$300.00,NaN,NaN,NaN,NaN,"$46,591.98",02118
1,"Abasciano,Joseph",Boston Police Department,Police Officer,"$6,933.66",NaN,$850.00,$205.92,"$74,331.86",NaN,"$15,258.44","$97,579.88",02132
2,"Abban,Christopher John",Boston Fire Department,Fire Fighter,"$103,442.22",NaN,$550.00,"$15,884.53",NaN,"$4,746.50",NaN,"$124,623.25",02132
3,"Abbasi,Sophia",Green Academy,Manager (C) (non-ac),"$18,249.83",NaN,NaN,NaN,NaN,NaN,NaN,"$18,249.83",02148
4,"Abbate-Vaughn,Jorgelina",BPS Ellis Elementary,Teacher,"$84,410.28",NaN,"$1,250.00",NaN,NaN,NaN,NaN,"$85,660.28",02481


In [52]:
df16.columns = df16.columns.str.lower()
df16.columns = df16.columns.str.strip()
df16.columns = df16.columns.str.replace(" ", "_")
df16.head()

,name,department_name,title,regular,retro,other,overtime,injured,detail,quinn/education_incentive,total_earnings,postal
0,"Abadi,Kidani A",Assessing Department,Property Officer,"$46,291.98",NaN,$300.00,NaN,NaN,NaN,NaN,"$46,591.98",02118
1,"Abasciano,Joseph",Boston Police Department,Police Officer,"$6,933.66",NaN,$850.00,$205.92,"$74,331.86",NaN,"$15,258.44","$97,579.88",02132
2,"Abban,Christopher John",Boston Fire Department,Fire Fighter,"$103,442.22",NaN,$550.00,"$15,884.53",NaN,"$4,746.50",NaN,"$124,623.25",02132
3,"Abbasi,Sophia",Green Academy,Manager (C) (non-ac),"$18,249.83",NaN,NaN,NaN,NaN,NaN,NaN,"$18,249.83",02148
4,"Abbate-Vaughn,Jorgelina",BPS Ellis Elementary,Teacher,"$84,410.28",NaN,"$1,250.00",NaN,NaN,NaN,NaN,"$85,660.28",02481


In [53]:
df17 = pd.read_csv("../input/2017.csv", encoding="latin1")
df17.head()

,NAME,DEPARTMENT NAME,TITLE,REGULAR,RETRO,OTHER,OVERTIME,INJURED,DETAIL,QUINN/EDUCATION INCENTIVE,TOTAL EARNINGS,POSTAL
0,"Miller,Francis T",Boston Police Department,Police Lieutenant (Det),"$129,530.83",NaN,"$13,694.50","$8,150.41",NaN,NaN,"$24,286.86","$175,662.60",2043
1,"Sullivan,Patrice Murphy",Boston Police Department,Office Manager,"$56,921.80",NaN,"$3,594.65","$1,548.16",NaN,NaN,NaN,"$62,064.61",2132
2,"O'Hara,Michael C",Boston Police Department,Police Lieutenant,"$124,056.64",NaN,"$6,432.23","$29,043.67",NaN,"$22,385.00","$23,260.69","$205,178.23",2021
3,"Whalen,Richard L",Boston Police Department,Police Detective,"$94,956.25","$4,985.32","$13,591.93","$85,419.22",$58.03,"$16,330.00","$19,971.01","$235,311.76",2062
4,"Kelly,Ann Marie",Boston Police Department,Tape Librarian I (BPD),"$69,995.34",NaN,$300.00,"$7,961.08",NaN,NaN,NaN,"$78,256.42",2190


In [54]:
df17.columns = df17.columns.str.lower()
df17.columns = df17.columns.str.strip()
df17.columns = df17.columns.str.replace(" ", "_")
df17.head()

,name,department_name,title,regular,retro,other,overtime,injured,detail,quinn/education_incentive,total_earnings,postal
0,"Miller,Francis T",Boston Police Department,Police Lieutenant (Det),"$129,530.83",NaN,"$13,694.50","$8,150.41",NaN,NaN,"$24,286.86","$175,662.60",2043
1,"Sullivan,Patrice Murphy",Boston Police Department,Office Manager,"$56,921.80",NaN,"$3,594.65","$1,548.16",NaN,NaN,NaN,"$62,064.61",2132
2,"O'Hara,Michael C",Boston Police Department,Police Lieutenant,"$124,056.64",NaN,"$6,432.23","$29,043.67",NaN,"$22,385.00","$23,260.69","$205,178.23",2021
3,"Whalen,Richard L",Boston Police Department,Police Detective,"$94,956.25","$4,985.32","$13,591.93","$85,419.22",$58.03,"$16,330.00","$19,971.01","$235,311.76",2062
4,"Kelly,Ann Marie",Boston Police Department,Tape Librarian I (BPD),"$69,995.34",NaN,$300.00,"$7,961.08",NaN,NaN,NaN,"$78,256.42",2190


In [55]:
df15_prepped = df15[df15.department_name == "Boston Police Department"]
df16_prepped = df16[df16.department_name == "Boston Police Department"]
df17_prepped = df17[df17.department_name == "Boston Police Department"]

In [56]:
df15_prepped["year"] = 2015
df16_prepped["year"] = 2016
df17_prepped["year"] = 2017

/var/folders/8w/c7bhb4y56693wcs88hywlmq00000gn/T/ipykernel_38774/3886895298.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df15_prepped["year"] = 2015
/var/folders/8w/c7bhb4y56693wcs88hywlmq00000gn/T/ipykernel_38774/3886895298.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df16_prepped["year"] = 2016
/var/folders/8w/c7bhb4y56693wcs88hywlmq00000gn/T/ipykernel_38774/3886895298.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [57]:
df15_17 = pd.concat([df15_prepped, df16_prepped, df17_prepped])
df15_17.to_csv("../data_prep/payroll15_17.csv", index=False)

In [58]:
df1 = pd.read_csv("../data_prep/payroll11_14.csv")
df2 = pd.read_csv("../data_prep/payroll15_17.csv")
df3 = pd.read_csv("../data_prep/payroll18_21.csv")

In [59]:
df1.head()

,name,department_name,title,regular,retro,other,overtime,injured,detail,quinn,total_earnings,zip,year
0,"Abasciano,Joseph",Boston Police Department,Police Officer,$68181.24,$0.00,$3763.50,$14177.65,$0.00,$4867.00,$0.00,$90989.39,02132-3802,2011
1,"Abdul-Aziz,Ramadani",Boston Police Department,Police Officer,$73935.02,$0.00,$800.00,$5573.40,$0.00,$44865.00,$0.00,$125173.42,02119-1618,2011
2,"Abichaker,Linda",Boston Police Department,Police Clerk And Typist,$34958.02,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$34958.02,02132-2737,2011
3,"Abreu,Cesar",Boston Police Department,Police Officer,$69701.19,$0.00,$800.00,$37800.40,$0.00,$29826.00,$0.00,$138127.59,02136-1540,2011
4,"Abron,Kim R.",Boston Police Department,School Traffic Supv,$10129.29,$0.00,$450.00,$0.00,$0.00,$0.00,$0.00,$10579.29,02124-4819,2011


In [60]:
df2.head()

,name,department_name,title,regular,retro,other,overtime,injured,detail,quinn/education_incentive,total_earnings,postal,year
0,"Abasciano,Joseph",Boston Police Department,Police Officer,$-886.20,$0.00,$8088.35,$544.83,$81996.78,$0.00,$20667.80,$110411.56,02132,2015
1,"Abdul-Aziz,Ramadani",Boston Police Department,Police Officer,$98860.29,$0.00,$2982.89,$24537.03,$0.00,$48426.00,$0.00,$174806.21,02119,2015
2,"Abichaker,Linda",Boston Police Department,Police Clerk And Typist,$45475.20,$0.00,$2500.00,$955.29,$0.00,$0.00,$0.00,$48930.49,02132,2015
3,"Abrahamson,Patrick Olaf",Boston Police Department,Police Officer,$73197.77,$0.00,$800.00,$2684.59,$0.00,$3374.00,$0.00,$80056.36,02135,2015
4,"Abreu,Cesar",Boston Police Department,Police Officer,$86331.52,$0.00,$800.00,$46469.48,$0.00,$21253.00,$0.00,$154854.00,02136,2015


In [61]:
df3.head()

,name,department_name,title,regular,retro,other,overtime,injured,detail,quinn/education_incentive,total_earnings,postal,year
0,"Williams,David C.",Boston Police Department,Police Officer,"95,184.68",0,"503,025.78","45,176.88","5,307.62","24,610.00","4,375.69","677,680.65",2356,2018
1,"Earley,John W",Boston Police Department,Police Lieutenant,"26,130.54","13,001.47","204,120.61",668.36,0,440.00,"6,532.63","250,893.61",2131,2018
2,"Bucelewicz,Helen",Boston Police Department,Police Officer,"8,676.89",0,"200,000.00",0,0,0,0,"208,676.89",2135,2018
3,"Evans,William B",Boston Police Department,Commissioner (Bpd),"149,999.93","3,461.56","165,858.44",0,0,0,0,"319,319.93",2127,2018
4,"Jones,Ronnie C",Boston Police Department,Police Officer,"462,149.98",0,"114,973.46",0,0,0,0,"577,123.44",2126,2018


In [62]:
df2.columns = df1.columns
df3.columns = df1.columns
df2.head()


,name,department_name,title,regular,retro,other,overtime,injured,detail,quinn,total_earnings,zip,year
0,"Abasciano,Joseph",Boston Police Department,Police Officer,$-886.20,$0.00,$8088.35,$544.83,$81996.78,$0.00,$20667.80,$110411.56,02132,2015
1,"Abdul-Aziz,Ramadani",Boston Police Department,Police Officer,$98860.29,$0.00,$2982.89,$24537.03,$0.00,$48426.00,$0.00,$174806.21,02119,2015
2,"Abichaker,Linda",Boston Police Department,Police Clerk And Typist,$45475.20,$0.00,$2500.00,$955.29,$0.00,$0.00,$0.00,$48930.49,02132,2015
3,"Abrahamson,Patrick Olaf",Boston Police Department,Police Officer,$73197.77,$0.00,$800.00,$2684.59,$0.00,$3374.00,$0.00,$80056.36,02135,2015
4,"Abreu,Cesar",Boston Police Department,Police Officer,$86331.52,$0.00,$800.00,$46469.48,$0.00,$21253.00,$0.00,$154854.00,02136,2015


In [63]:
df3.head()

,name,department_name,title,regular,retro,other,overtime,injured,detail,quinn,total_earnings,zip,year
0,"Williams,David C.",Boston Police Department,Police Officer,"95,184.68",0,"503,025.78","45,176.88","5,307.62","24,610.00","4,375.69","677,680.65",2356,2018
1,"Earley,John W",Boston Police Department,Police Lieutenant,"26,130.54","13,001.47","204,120.61",668.36,0,440.00,"6,532.63","250,893.61",2131,2018
2,"Bucelewicz,Helen",Boston Police Department,Police Officer,"8,676.89",0,"200,000.00",0,0,0,0,"208,676.89",2135,2018
3,"Evans,William B",Boston Police Department,Commissioner (Bpd),"149,999.93","3,461.56","165,858.44",0,0,0,0,"319,319.93",2127,2018
4,"Jones,Ronnie C",Boston Police Department,Police Officer,"462,149.98",0,"114,973.46",0,0,0,0,"577,123.44",2126,2018


In [64]:
payroll = pd.concat([df1, df2, df3])
payroll.head()

,name,department_name,title,regular,retro,other,overtime,injured,detail,quinn,total_earnings,zip,year
0,"Abasciano,Joseph",Boston Police Department,Police Officer,$68181.24,$0.00,$3763.50,$14177.65,$0.00,$4867.00,$0.00,$90989.39,02132-3802,2011
1,"Abdul-Aziz,Ramadani",Boston Police Department,Police Officer,$73935.02,$0.00,$800.00,$5573.40,$0.00,$44865.00,$0.00,$125173.42,02119-1618,2011
2,"Abichaker,Linda",Boston Police Department,Police Clerk And Typist,$34958.02,$0.00,$0.00,$0.00,$0.00,$0.00,$0.00,$34958.02,02132-2737,2011
3,"Abreu,Cesar",Boston Police Department,Police Officer,$69701.19,$0.00,$800.00,$37800.40,$0.00,$29826.00,$0.00,$138127.59,02136-1540,2011
4,"Abron,Kim R.",Boston Police Department,School Traffic Supv,$10129.29,$0.00,$450.00,$0.00,$0.00,$0.00,$0.00,$10579.29,02124-4819,2011


In [65]:
payroll.to_csv("../data_prep/payroll11_21.csv", index=False)